## Encoding Rewards

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Encoding Rewards

- We've now discussed the importance of encoding the observations.
- We may also have some choice on the action space, though here (and often) it is relatively clear/fixed.
- But what about the rewards? 

#### Current set-up

- Currently, we get a reward of +1 for reaching the goal. 
- This is part of what makes RL so hard (and impressive):
  - We want to learn about actions even though we don't know right away whether the action was "good". 
  - Contrast this with supervised learning, where every prediction we make on the training data can immediately be compared with the known target value.


In [2]:
# TODO: perhaps this next slide can be moved to Module 1, since it's very general?

#### Agents can't just be greedy

- Can agents simply learn to go for the best immediate reward?
- No. For example, in a video recommendation system, showing the user another funny cat video might make them click (high immediate reward) but result in long-term loss of interest in the service (low long-term reward).
- Our Frozen Lake is another example of the problem here: sometimes there is no immediate reward at all to learn from.

In [3]:
# TODO: perhaps this next section on "Learned action probabilities" could be moved much earlier, even as early as Module 1

#### Learned action probabilities

- RLlib lets us look inside the model at the probability of each action given an observation (i.e., the learned policy).
- Let's load the trained model with our encoded observations:

In [4]:
from envs import RandomLakeObs
from ray.rllib.algorithms.ppo import PPOConfig

ppo_config = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")
)
ppo_RandomLakeObs = ppo_config.build(env=RandomLakeObs)

In [11]:
# # HIDDEN

# for i in range(8):
#     ppo_RandomLakeObs.train()
    
# print(ppo_RandomLakeObs.evaluate()["evaluation"]["episode_reward_mean"])

# ppo_RandomLakeObs.save("models/RandomLakeObs")

In [19]:
ppo_RandomLakeObs.restore("models/RandomLakeObs/checkpoint-16")

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

#### Learned action probabilities

The exact RLlib code is hidden inside `utils` for now, but we're mainly interested in the results:

In [20]:
from utils import query_policy
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,0,0,0])

array([0.02463268, 0.45315894, 0.49981508, 0.02239344], dtype=float32)

- Recall the (left, down, right, up) ordering.
- When the observation is `[0 0 0 0]` (no holes or edges in sight), the agent prefers to go down and right.

What if there's a hole below you? We can feed in a different observation to the policy:

In [21]:
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,1,0,0])

array([0.03616549, 0.03877434, 0.9037791 , 0.02128115], dtype=float32)

- Now the agent is very unlikely to go down, and very likely to go right!
- Again, all this was learned from trial and error, with a reward earned only when the goal was reached.

#### Random Lake rewards

- In the Random Lake example, can't be make life easier for the agent by giving immediate rewards?

This is the current reward code:

In [22]:
def reward(self):
    return int(self.player == self.goal)

- The agent has to learn, through trial and error over _entire episodes_, that moving down and right is generally a good thing. 

#### Redefining rewards

- Let's instead try giving a reward _at every step, that is higher as the agent gets closer to the goal_. 

In [23]:
from envs import RandomLakeObs

class RandomLakeObsRew(RandomLakeObs):
    def reward(self):
        return 6-(abs(self.player[0]-self.goal[0]) + abs(self.player[1]-self.goal[1]))

- The above method uses the [Manhattan Distance](https://en.wikipedia.org/wiki/Taxicab_geometry) between the player and the goal as the reward. 
- When the agent reaches the goal, the maximum reward of 6 is achieved.
- When the agent is furthest from the goal, the minimum reward of 0 is given.

#### Redefining rewards

In [24]:
env = RandomLakeObsRew()
env.reset()
env.render()
env.reward()

🧑🕳🧊🕳
🧊🧊🧊🕳
🕳🕳🧊🧊
🧊🧊🕳⛳️


0

⬆️ the reward is 0

⬇️ the reward is 1 because we moved closer to the goal

In [25]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🕳🧊🕳
🧑🧊🧊🕳
🕳🕳🧊🧊
🧊🧊🕳⛳️


1

#### Redefining rewards

In [26]:
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🕳🧊🧑
🧊🧊🕳⛳️


5

Now, the reward is 5. Next, it will be 6.

In [27]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🕳🧊🧊
🧊🧊🕳🧑


6

#### Comparing rewards

- So, we have two possible reward functions. Which one works better? 
- Recall that last time, after training for 8 iterations, we were able to reach the goal around 70% of the time:

In [28]:
ppo_RandomLakeObs.evaluate()['evaluation']['episode_reward_mean']

0.7130434782608696

#### Comparing rewards

Let's train with the new reward function!

In [32]:
ppo_RandomLakeObsRew = ppo_config.build(env=RandomLakeObsRew)

In [33]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [34]:
ppo_RandomLakeObsRew.evaluate()['evaluation']['episode_reward_mean']

101.90566037735849

Wait a minute, what's going on here??

#### Comparing rewards?

- We tried to improve our RL system by shaping the reward function.
- This (presumably) affected training, but it also affected our evaluation.
- In supervised learning, this is like changed the scoring metric from squared error to absolute error.
- If the old system got a mean squared error of 20,000 and the new system got a mean absolute error of 40, which is better?
- We're comparing apples and oranges here!
- We want to compare both models on the same metric, for example the original metric. 
- Here, we want to see how frequently the agent reaches the goal.

#### Comparing rewards?

- The code here is a bit more advanced.
- It is included for completeness, but we won't go into detail.

In [35]:
from ray.rllib.agents.callbacks import DefaultCallbacks

class MyCallbacks(DefaultCallbacks):
    def on_episode_end(self, *, worker, base_env, policies, episode, env_index, **kwargs):
        info = episode.last_info_for()
        episode.custom_metrics["goal_reached"] = info["player"] == info["goal"]

In [36]:
ppo_config_callback = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")\
    .callbacks(callbacks_class=MyCallbacks)\
    .evaluation(evaluation_config={"callbacks" : MyCallbacks})
)

ppo_RandomLakeObsRew = ppo_config_callback.build(env=RandomLakeObsRew)

The trainer above uses our new reward scheme but also reports/measures the rate of reaching the goal.

#### Comparing rewards?

Let's try it out!

In [37]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [38]:
# HIDDEN
ppo_RandomLakeObsRew.evaluate()["evaluation"]["episode_reward_mean"]

101.90566037735849

In [39]:
ppo_RandomLakeObsRew.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.04081632653061224

- Hmm, these results are terrible!
- We used to get a 70%+ win rate, and now we're close to zero.
- What happened? 🤔

#### What is the agent really optimizing?

- The agent is really optimizing _discounted total reward_.
- _Total_: it values all the rewards it collects, not just the final reward.
- _Discounted_: it values earlier rewards more than later ones.
- Our agent is successfully maximizing discounted total reward, but this isn't corresponding to reaching the goal.
- But why? The goal gives a higher reward.

#### Exploration vs. exploitation

- A fundamental concept in RL is _exploration vs. exploitation_
- When the agent is learning the policy, it can choose to either:

1. Do things that it knows are pretty good ("exploit")
2. Try something totally new and crazy, just in case ("explore")

In [ ]:
# TODO 
# diagram for this?

#### Exploration vs. exploitation

- With the old reward structure, the agent gets a reward of 0 unless it reaches the goal.
  - So, it keeps trying to find something better.
- With the new reward structure, the agent is getting lots of reward just for walking around.
  - It isn't very motivated to explore the environment.
- In fact, because it is maximizing discounted **total** reward, finding the goal is a bad thing!
  - This causes the episode to end, limiting the total reward of the agent.
  - The agent actually learns to _avoid_ the goal, especially early on in the episode.

#### Designing a better reward structure

- Instead, let's try penalizing the agent when it walks into a hole or off the edge.
- It will be easier to implement this directly in `step`:

In [40]:
class RandomLakeObsRew2(RandomLakeObs):
    def step(self, action):
        # (not shown) existing code gets new_loc, where the player is trying to go
        
        reward = 0
        
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        else:
            reward -= 0.1 # small penalty
            
        if self.holes[self.player]:
            reward -= 0.1 # small penalty
            
        if self.player == self.goal:
            reward += 1
        
        # Return observation/reward/done
        return self.observation(), reward, self.done(), {"player" : self.player, "goal" : self.goal}

In [41]:
# HIDDEN
from envs import RandomLakeObsRew2

#### Testing it out, again

In [ ]:
# HIDDEN
# redefine ppo_RandomLakeObs to include the new callbacks
# so that you can measure the custom metric instead of the reward
# they will give the same value but this is better for consistency
ppo_RandomLakeObs = ppo_config_callback.build(env=RandomLakeObs)

for i in range(8):
    ppo_RandomLakeObs.train()

In [ ]:
ppo_config_callback.build(env=RandomLakeObsRew2)

In [ ]:
for i in range(8):
    ppo_RandomLakeObsRew2.train()

In [ ]:
ppo_RandomLakeObs.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

In [ ]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

It looks like, this time, the two methods perform about the same. 

#### Episode length

- In addition to the success rate, we can compute other statistics of the agent's behavior.
- One interesting measure is episode length.
- RLlib records this by default, so we can easily access it:

In [ ]:
ppo_RandomLakeObs.evaluate()["evaluation"]["episode_len_mean"]

In [ ]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["episode_len_mean"]

- Although the two agents have the same success rate, the new one tends toward shorter episodes.
- This is quite interesting because the agent cannot "see" the difference between holes and edges.
- We could explore this further by adding more custom metrics, e.g. number of bumps into the edge.

In [ ]:
# TODO
#### disadvantages - loss of generality

#- now only works if goal is at bottom-right
#give a few real-world examples here -> important

## Supervised learning analogy: reward shaping
<!-- multiple choice -->

Earlier, we made an analogy between encoding observations in RL and feature preprocessing in supervised learning. What aspect of supervised learning is the best analogy to reward shaping in RL?

- [ ] Feature engineering 
- [ ] Model selection | Not quite. But, as we'll see, there is a place for model selection in RL as well!
- [ ] Hyperparameter tuning | Not quite. But, as we'll see, there is a place for hyperparameter tuning in RL as well!
- [x] Selecting a loss function | Changing the loss function changes the "best" model, just like changing the rewards changes the "best" policy.

## Rewarding every step
<!-- multiple choice -->

#### Small negative rewards

In RL environments like Random Lake where the agent needs to reach a specific goal, imagine we assigned a tiny negative reward for _every_ step taken by the agent. How would this generally/typically affect the amount of time the agent spends until it reaches the goal?

- [x] The agent will try to reach the goal in as few steps as possible.
- [ ] The agent will try to reach the goal in as many steps as possible. | If we're penalizing each step, taking more steps will result in less reward.
- [ ] No change. | If we're penalizing each step, taking more steps will result in less reward.

## Unintended consequences
<!-- coding exercise -->

In this exercise, you will try out a bad idea: assigning a large negative reward every time the agent takes a step. We will use -1 per step. The agent still gets a reward of +1 for reaching the goal. Implement this reward, train the agent, and look at the average episode length printed out by the code. Compare this to the average episode length of an agent that just acts randomly. Then, answer the multiple choice question about the agent's behavior. What do you think is going on here? 

In [ ]:
# EXERCISE
from utils import lake_default_config
from envs import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return ____
    
ppo = lake_default_config.build(env=____)

for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"][____])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"][____])

In [ ]:
# SOLUTION
from utils import lake_default_config
from envs import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return old_reward - 1

ppo = lake_default_config.build(env=RandomLakeBadIdea)


for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      trainer.evaluate()["evaluation"]["episode_len_mean"])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"]["episode_len_mean"])

#### Agent's behavior

When trained on an environment with a large negative reward at every step, what do you think this agent is doing, that is undesirable?

- [ ] The agent stays still because it is be discouraged from moving. | Try again!
- [ ] The agent is not interested in reaching the goal because the reward is comparatively small. | Try again!
- [x] The agent learns to jump into the lake as fast as it can, to avoid the negative reward of moving. | Yikes! 🥶
- [ ] The agent reaches the goal right away. | That would be desirable though!